In [56]:
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import numpy as np
from sklearn.decomposition import PCA
import statistics
import matplotlib.pyplot as plt
from sklearn import preprocessing
import category_encoders as ce
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score, StratifiedKFold, cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

In [57]:
class BinaryAndMulticlassClassifier(BaseEstimator, TransformerMixin):
    def __init__(self, binary_classifier, multiclass_classifier):
        self.binary_classifier = binary_classifier
        self.multiclass_classifier = multiclass_classifier

    def fit(self, X, y):
        # Binary classification fit
        self.binary_classifier.fit(X, np.where(y == 3, 1, 0))
        
        # Apply binary classifier
        binary_predictions = self.binary_classifier.predict(X)
        
        # Prepare data for multiclass classification
        X_not_class_3 = X[binary_predictions == 0]
        y_not_class_3 = y[binary_predictions == 0]
        
        # Multiclass classification fit
        self.multiclass_classifier.fit(X_not_class_3, y_not_class_3)
        return self

    def predict(self, X):
        binary_predictions = self.binary_classifier.predict(X)
        X_not_class_3 = X[binary_predictions == 0]
        
        multiclass_predictions = np.full(X.shape[0], -1)  # Initialize with default class
        multiclass_predictions[binary_predictions == 0] = self.multiclass_classifier.predict(X_not_class_3)
        return np.where(binary_predictions == 1, 3, multiclass_predictions)


In [58]:
df = pd.read_csv("Encoded_data.csv")

In [59]:
df.drop("Unnamed: 0", axis = 1, inplace = True)

In [60]:
df["status"].value_counts()

status
3    54878
0     3457
1     1511
2      429
Name: count, dtype: int64

In [61]:
X = df.drop(['status'], axis=1)
y = df['status']

In [62]:
ros = RandomOverSampler(random_state=42)
X_res, y_res = ros.fit_resample(X, y)

print(f"Original dataset shape: {Counter(y)}")
print(f"Resampled dataset shape: {Counter(y_res)}")

Original dataset shape: Counter({3: 54878, 0: 3457, 1: 1511, 2: 429})
Resampled dataset shape: Counter({3: 54878, 0: 54878, 1: 54878, 2: 54878})


In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [64]:
y_train_binary = np.where(y_train == 3, 1, 0)

In [65]:
binary_pipeline = Pipeline([
    ('scaler', StandardScaler()),  
    #('pca', PCA(n_components=10)),
    ('binary_classifier', LogisticRegression(penalty='l1', solver='saga', max_iter=50, multi_class='multinomial', C=1))
])

In [66]:
binary_pipeline.fit(X_train, y_train_binary)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Pipeline(steps=[('scaler', StandardScaler()),
                ('binary_classifier',
                 LogisticRegression(C=1, max_iter=50, multi_class='multinomial',
                                    penalty='l1', solver='saga'))])

In [67]:
binary_predictions = binary_pipeline.predict(X_test)

In [68]:
X_test_not_class_3 = X_test[binary_predictions == 0]
y_test_not_class_3 = y_test[binary_predictions == 0]

In [69]:
X_train_not_class_3 = X_train[y_train != 3]
y_train_not_class_3 = y_train[y_train != 3]

In [70]:
multiclass_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    #('pca', PCA(n_components=10)),
    ('multiclass_classifier', RandomForestClassifier(random_state=42))
])


In [71]:
multiclass_pipeline.fit(X_train_not_class_3, y_train_not_class_3)

Pipeline(steps=[('scaler', StandardScaler()),
                ('multiclass_classifier',
                 RandomForestClassifier(random_state=42))])

In [72]:
combined_pipeline = Pipeline([
    ('model', BinaryAndMulticlassClassifier(binary_pipeline, multiclass_pipeline))
])

# Fit the combined pipeline
combined_pipeline.fit(X_train, y_train)

# Predict with the combined pipeline
final_predictions = combined_pipeline.predict(X_test)
print(f"Final predictions: {Counter(final_predictions)}")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Final predictions: Counter({3: 11046, 0: 705, 1: 297, 2: 7})


In [73]:
#multiclass_predictions = multiclass_pipeline.predict(X_test_not_class_3)

In [74]:
#final_predictions = binary_predictions.copy()

In [75]:
#final_predictions[binary_predictions == 0] = multiclass_predictions

In [76]:
#print("Final predictions:", final_predictions)

In [77]:
accuracy_score(y_test, final_predictions)

0.9912069680630444

In [78]:
import dill

with open('combined_plzz.pkl', 'wb') as file:
    dill.dump(combined_pipeline, file)

In [79]:
import joblib

# Save the model
joblib.dump(combined_pipeline, 'combined_yr.pkl')

['combined_yr.pkl']